In [1]:
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
import time
from IPython.display import display

In [10]:
#pip install requests beautifulsoup4 folium geopy

     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
   ---------------------------------------- 125.4/125.4 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Path to your WebDriver (e.g., ChromeDriver)
webdriver_path = 'D:/s9/Concertify/chromedriver-win64/chromedriver.exe'

# Initialize a Selenium WebDriver
driver = webdriver.Chrome(webdriver_path)

base_url = "https://www.ticketmaster.fr"

# Go to the Ticketmaster Paris page
driver.get('https://www.ticketmaster.fr/fr/paris')
time.sleep(5)  # Wait for JavaScript to load the content

# Get the page source and close the browser
html = driver.page_source
driver.quit()

# Parse the page with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

concerts = []
for i in range(20):
    event = soup.find('li', id=f'event-result-{i}')
    if event:
        title = event.find('h3', class_='event-result-title').get_text(strip=True)
        date = event.find('time', itemprop='startDate').get_text(strip=True)

        # Extract the arena name
        arena = event.find('span', itemprop='name').get_text(strip=True) if event.find('span', itemprop='name') else "Unknown Arena"

        # Extract the city location
        city_location = event.find('span', itemprop='addressLocality').get_text(strip=True) if event.find('span', itemprop='addressLocality') else "Unknown City"

        # Extract the event URL
        event_link = event.find('a', class_='event-result-title-link')
        url = base_url + event_link['href'] if event_link and event_link.has_attr('href') else "URL not found"

        location = f"{arena}, {city_location}"
        concerts.append({"title": title, "location": location, "date": date, "event_url": url})


# Print the concert information
for concert in concerts:
    print(concert)


{'title': 'KAROL G', 'location': 'ACCOR ARENA, PARIS', 'date': '22 juin 2024', 'event_url': 'https://www.ticketmaster.fr/fr/manifestation/karol-g-billet/idmanif/570897'}
{'title': 'PACKAGE KAROL G', 'location': 'ACCOR ARENA, PARIS', 'date': '22 juin 2024', 'event_url': 'https://www.ticketmaster.fr/fr/pack/package-karol-g-billet/idpack/3356'}
{'title': 'PACKAGE KAROL G', 'location': 'ACCOR ARENA, PARIS', 'date': '23 juin 2024', 'event_url': 'https://www.ticketmaster.fr/fr/pack/package-karol-g-billet/idpack/3385'}
{'title': 'SLIPKNOT', 'location': 'ACCOR ARENA, PARIS', 'date': '12 décembre 2024', 'event_url': 'https://www.ticketmaster.fr/fr/manifestation/slipknot-billet/idmanif/571645'}
{'title': 'SLIPKNOT', 'location': 'ACCOR ARENA, PARIS', 'date': '12 décembre 2024', 'event_url': 'https://www.ticketmaster.fr/fr/pack/slipknot-billet/idpack/3366'}
{'title': 'DOJA CAT', 'location': 'ACCOR ARENA, PARIS', 'date': '21 juin 2024', 'event_url': 'https://www.ticketmaster.fr/fr/manifestation/doj

In [7]:
def geocode_location(location):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        loc = geolocator.geocode(location)
        return (loc.latitude, loc.longitude)
    except:
        return None

# Geocode each concert location
for concert in concerts:
    coords = geocode_location(concert['location'])
    if coords:
        concert['coords'] = coords
    else:
        concert['coords'] = (0, 0)  # Default coords, adjust as necessary

# Group concerts by location
concerts_by_location = {}
for concert in concerts:
    key = concert['coords']
    if key not in concerts_by_location:
        concerts_by_location[key] = {'name': concert['location'], 'concerts': []}
    concerts_by_location[key]['concerts'].append(concert)

# Create a map and add markers
map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)  # Centered on Paris

for location, data in concerts_by_location.items():
    location_name = data['name']
    concerts_at_location = data['concerts']

    popup_content = f"<h3>{location_name}</h3><ul>"
    for concert in concerts_at_location:
        # Create a hyperlink for each concert
        concert_link = f"<a href='{concert['event_url']}' target='_blank'>{concert['title']}</a>"
        popup_content += f"<li>{concert_link} - {concert['date']}</li>"
    popup_content += '</ul>'

    # Use BeautifulSoup to format HTML in the popup
    soup = BeautifulSoup(popup_content, 'html.parser')
    popup_content = str(soup)

    folium.Marker(
        location=location,
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=location_name
    ).add_to(map)

# Add layer control and display the map
folium.LayerControl().add_to(map)
display(map)